# Saquemos temas de tweets

In [1]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rinco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Función que lematiza y hace el preprocesamintos del conjunto de datos:

In [2]:
from nltk.stem import PorterStemmer 

def lemmatize_stemming(text):
    ps = PorterStemmer()
    return ps.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text): #  gensim.utils.simple_preprocess tokeniza el texto
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [42]:
import pandas as pd
data = pd.read_csv('Tweetstr.csv', error_bad_lines=False);
data_text = data[['Tweet']]
data_text['index']=data_text.index
documents = data_text

<ipython-input-42-6fd6b18b14fe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index']=data_text.index


In [43]:
documents

,Tweet,index
0,{'translation_text': 'Gustavo Petro gives his ...,0
1,{'translation_text': 'The people where they pu...,1
2,{'translation_text': 'What is behind the winte...,2
3,{'translation_text': 'This is the map of areas...,3
4,{'translation_text': '‘The Democratic Center w...,4
...,...,...
2835,{'translation_text': 'Direct hiring in Casa de...,2835
2836,{'translation_text': 'Another scandal in Casa ...,2836
2837,{'translation_text': 'This trailer impacts as ...,2837
2838,{'translation_text': '#InColombiaTheChurchesTh...,2838


In [44]:
print(len(documents))
print(documents[:5])

2840
                                               Tweet  index
0  {'translation_text': 'Gustavo Petro gives his ...      0
1  {'translation_text': 'The people where they pu...      1
2  {'translation_text': 'What is behind the winte...      2
3  {'translation_text': 'This is the map of areas...      3
4  {'translation_text': '‘The Democratic Center w...      4


In [47]:
doc_sample = documents[documents['index'] == 2839].values[0][0]
print('documento original: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n documento tokenizado y lematizado: ')
print(preprocess(doc_sample))

documento original: 
["{'translation_text':", "'Says", 'María', 'Paula', 'Correa', 'that', 'she', 'was', '“assaulted', 'in', 'her', 'good', 'faith,”', 'those', 'who', 'are', 'assaulted', 'in', 'our', 'good', 'faith', 'are', 'Colombians,', 'every', "day.'}"]


 documento tokenizado y lematizado: 
['say', 'maría', 'paula', 'correa', 'assault', 'good', 'faith', 'assault', 'good', 'faith', 'colombian']


In [20]:
processed_docs = documents['Tweet'].map(preprocess)
processed_docs[:10]

0    [gustavo, petro, give, version, past, drink, s...
1    [peopl, rarest, name, world, newborn, http, ci...
2    [winter, tragedi, leav, dead, risaralda, tell,...
3    [area, risk, elector, campaign, http, pvfejcjq...
4    [democrat, center, walk, like, crazi, iron, ci...
5    [author, dosquebrada, declar, public, calam, s...
6    [cartoon, matador, wednesday, entitl, need, su...
7    [discoveri, steal, motorcycl, dismantl, entir,...
8    [costochondr, pain, symptom, covid, infect, ht...
9    [pereira, stori, hear, midst, relief, agenc, c...
Name: Tweet, dtype: object

In [13]:

dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 dgmnjt
1 drink
2 girardot
3 give
4 gustavo
5 http
6 past
7 petro
8 say
9 speech
10 umfsru


In [21]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

gensim doc2bow

In [48]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[2839]

[(4, 1), (48, 1), (292, 2)]

In [50]:
bow_doc_2839 = bow_corpus[2839]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_2839[i][1]))

Word 4 ("say") appears 1 time.
Word 48 ("colombian") appears 1 time.
Word 292 ("good") appears 2 time.


In [51]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5445638877127057),
 (1, 0.43340558479484914),
 (2, 0.47997580026243186),
 (3, 0.3721987965465388),
 (4, 0.3830157415558857)]


In [54]:
lda_model

In [53]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [61]:
processed_docs[2839]

['say',
 'maría',
 'paula',
 'correa',
 'assault',
 'good',
 'faith',
 'assault',
 'good',
 'faith',
 'colombian']

In [64]:
for index, score in sorted(lda_model[bow_corpus[2839]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.819962739944458	 
Topic: 0.038*"bogotá" + 0.022*"know" + 0.022*"colombia" + 0.017*"good" + 0.017*"today" + 0.016*"woman" + 0.015*"work" + 0.014*"great" + 0.012*"time" + 0.012*"polit"

Score: 0.020007213577628136	 
Topic: 0.034*"thank" + 0.027*"today" + 0.018*"colombian" + 0.017*"look" + 0.016*"tell" + 0.015*"right" + 0.015*"stori" + 0.014*"leav" + 0.013*"peopl" + 0.013*"year"

Score: 0.02000497654080391	 
Topic: 0.024*"colombia" + 0.023*"candid" + 0.020*"petro" + 0.016*"urib" + 0.016*"senat" + 0.015*"film" + 0.014*"pact" + 0.013*"campaign" + 0.013*"countri" + 0.012*"state"

Score: 0.020004529505968094	 
Topic: 0.029*"elect" + 0.025*"presid" + 0.019*"losdaniel" + 0.017*"covid" + 0.015*"vote" + 0.015*"year" + 0.014*"gener" + 0.014*"call" + 0.013*"talk" + 0.012*"elpaisenvivo"

Score: 0.02000439539551735	 
Topic: 0.025*"like" + 0.019*"colombia" + 0.014*"best" + 0.013*"leav" + 0.013*"chang" + 0.013*"follow" + 0.012*"commun" + 0.012*"govern" + 0.012*"leticia" + 0.011*"polit"

Score